In [ ]:
# So when making the plan, the agent should first determine the broad sketches
# of the day, then fill in the details, and finally create the calendar events.
# in creating the calendar events, it should make sure to place them relativily,
# put some breathing room between them, and then generate the individual items 
'''
<calendarEvents>
  <!-- Morning Routine -->
  <calendarEvent>
    <title>Wake Up</title>
    <dtstart>2025-07-22T07:30:00</dtstart>
    <dtend>2025-07-22T07:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Morning Routine (Breakfast, Prep)</title>
    <dtstart>2025-07-22T07:30:00</dtstart>
    <dtend>2025-07-22T08:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Cycle to Office</title>
    <dtstart>2025-07-22T08:30:00</dtstart>
    <dtend>2025-07-22T09:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Day-Ahead Review</title>
    <dtstart>2025-07-22T09:00:00</dtstart>
    <dtend>2025-07-22T09:30:00</dtend>
  </calendarEvent>

  <!-- First Deep Work & Break -->
  <calendarEvent>
    <title>DeepWork: C2T – Partitional Dissonance</title>
    <dtstart>2025-07-22T09:30:00</dtstart>
    <dtend>2025-07-22T11:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Rejuvenation Break (Sci-Fi Reading)</title>
    <dtstart>2025-07-22T11:30:00</dtstart>
    <dtend>2025-07-22T12:00:00</dtend>
  </calendarEvent>

  <!-- FateForger Deep Work (with Lunch) -->
  <calendarEvent>
    <title>DeepWork: FateForger – Calendar Agent</title>
    <dtstart>2025-07-22T12:00:00</dtstart>
    <dtend>2025-07-22T14:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Lunch</title>
    <dtstart>2025-07-22T12:30:00</dtstart>
    <dtend>2025-07-22T13:00:00</dtend>
  </calendarEvent>

  <!-- Pre-Second Block Prep -->
  <calendarEvent>
    <title>Meditation (10 min)</title>
    <dtstart>2025-07-22T14:00:00</dtstart>
    <dtend>2025-07-22T14:10:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Buffer / Light Tasks</title>
    <dtstart>2025-07-22T14:10:00</dtstart>
    <dtend>2025-07-22T15:00:00</dtend>
  </calendarEvent>

  <!-- Second Deep Work -->
  <calendarEvent>
    <title>DeepWork: C2T – Gerimedica Verslaglegging</title>
    <dtstart>2025-07-22T15:00:00</dtstart>
    <dtend>2025-07-22T17:00:00</dtend>
  </calendarEvent>

  <!-- Quick Shutdown Rituals -->
  <calendarEvent>
    <title>TickTick Deep Clean</title>
    <dtstart>2025-07-22T17:00:00</dtstart>
    <dtend>2025-07-22T17:15:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Planning Session (Next Day)</title>
    <dtstart>2025-07-22T17:15:00</dtstart>
    <dtend>2025-07-22T17:30:00</dtend>
  </calendarEvent>

  <!-- Gym & Commute -->
  <calendarEvent>
    <title>Gym</title>
    <dtstart>2025-07-22T17:30:00</dtstart>
    <dtend>2025-07-22T19:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Cycle Home</title>
    <dtstart>2025-07-22T19:00:00</dtstart>
    <dtend>2025-07-22T19:30:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Log Workout in Strong App</title>
    <dtstart>2025-07-22T19:30:00</dtstart>
    <dtend>2025-07-22T19:40:00</dtend>
  </calendarEvent>

  <!-- Evening Shutdown Ritual Subtasks -->
  <calendarEvent>
    <title>Dog Walk</title>
    <dtstart>2025-07-22T22:00:00</dtstart>
    <dtend>2025-07-22T22:10:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Pack Bag &amp; Clothes</title>
    <dtstart>2025-07-22T22:10:00</dtstart>
    <dtend>2025-07-22T22:15:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Clean Face &amp; Brush Teeth</title>
    <dtstart>2025-07-22T22:15:00</dtstart>
    <dtend>2025-07-22T22:25:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Evening Reading</title>
    <dtstart>2025-07-22T22:25:00</dtstart>
    <dtend>2025-07-22T23:00:00</dtend>
  </calendarEvent>
  <calendarEvent>
    <title>Sleep</title>
    <dtstart>2025-07-22T23:00:00</dtstart>
    <dtend>2025-07-23T07:30:00</dtend>
  </calendarEvent>
</calendarEvents>
'''

In [1]:
import logging
import sys

# 1) Reset any existing handlers and force new config (Python ≥3.8)
logging.root.handlers.clear()
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(name)s %(levelname)s %(message)s",
    force=True,           # override any prior config
    stream=sys.stdout,
)

# 2) Names from the logging docs
from autogen_core import TRACE_LOGGER_NAME as CORE_TRACE, EVENT_LOGGER_NAME as CORE_EVENT
from autogen_agentchat import TRACE_LOGGER_NAME as CHAT_TRACE, EVENT_LOGGER_NAME as CHAT_EVENT

# 3) Explicitly enable each
for logger_name in (
    CORE_TRACE,
    CORE_EVENT,
    CHAT_TRACE,
    CHAT_EVENT,
):
    lg = logging.getLogger(logger_name)
    lg.setLevel(logging.DEBUG)
    if not lg.handlers:
        lg.addHandler(logging.StreamHandler(sys.stdout))

# 4) Finally, also turn on any “root” AutoGen logs not covered above
logging.getLogger("autogen_core").setLevel(logging.DEBUG)
logging.getLogger("autogen_agentchat").setLevel(logging.DEBUG)

In [2]:
# Test the agent with timeout to prevent hanging
import asyncio

# Let's test with a timeout to prevent hanging
async def test_agent_with_timeout():
    # Restart the kernel to get fresh agent code
    import importlib
    import sys

    # Clear the agent module from cache to reload changes
    modules_to_clear = [
        'fateforger.agents.schedular.agent',
        'fateforger.core.runtime',
        'tools_config'
    ]
    for module in modules_to_clear:
        if module in sys.modules:
            del sys.modules[module]

    from fateforger.core.runtime import runtime, initialize_runtime
    from fateforger.core.config import settings

    from autogen_core import AgentId

    # Check configuration first
    print(f"OpenAI API Key configured: {settings.openai_api_key[:10]}..." if len(settings.openai_api_key) > 10 else f"OpenAI API Key: {settings.openai_api_key}")

    # Let's also import the Message class that our agent expects
    from fateforger.agents.schedular.agent import Message

    print("Initializing runtime with fresh agent code...")

    try:
        # This should create a fresh runtime with the updated agent code
        runtime = await asyncio.wait_for(initialize_runtime(), timeout=10.0)
        print("Runtime initialized successfully!")
        
        print("Sending message to planner agent...")
        
        # Use the Message class that our agent is configured to handle  
        response = await asyncio.wait_for(
            runtime.send_message(
                Message(content="What meetings do I have tomorrow?"),
                recipient=AgentId("planner_agent","default")
            ),
            timeout=15.0  # 15 second timeout
        )
        
        print(f"Response received: {response}")
        print(f"Response type: {type(response)}")
        if hasattr(response, 'content'):
            print(f"Response content: {response.content}")
            
    except asyncio.TimeoutError:
        print("TIMEOUT: Agent took too long to respond! This confirms it's hanging.")
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

    print("Test complete!")

# Run the test
await test_agent_with_timeout()

OpenAI API Key configured: sk-proj-UD...
Initializing runtime with fresh agent code...
Runtime initialized successfully!
Sending message to planner agent...
TIMEOUT: Agent took too long to respond! This confirms it's hanging.
Test complete!
TIMEOUT: Agent took too long to respond! This confirms it's hanging.
Test complete!


In [ ]:
from fateforger.core.bootstrap import runtime
from fateforger.models.events import CalendarEvent
from autogen_core import AgentId

# all agents should be pre-registered
runtime.start()

await runtime.send_message(
    CalendarEvent(summary="Demo", start={...}, end={...}),
    recipient=AgentId("planner","default")      # LLM decides tool
)
# how do i get the message out?
await asyncio.sleep(1)
await runtime.stop()

In [ ]:
# TODO: add the slack send tool to the chain later